In [1]:
import pandas as pd
import numpy as np
import requests
import re

from bs4 import BeautifulSoup
import locale
import time
locale.setlocale(locale.LC_ALL, 'nl_NL.UTF-8')

import aux

from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
pd.read_table("../data/trial_small/metadata_in.csv", index_col = 0).head(5)

,Instantie,Datum uitspraak,Datum publicatie,Zaaknummer,Rechtsgebieden,Bijzondere kenmerken,Inhoudsindicatie,Vindplaatsen,Unnamed: 9,Wetsverwijzingen,Formele relaties
0,Rechtbank Den Haag,09-11-2022,09-11-2022,C-09-632745-KG ZA 22-669,Civiel recht,Kort geding,Kort geding. Vordering om Staat te verbieden e...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
1,Rechtbank Den Haag,09-11-2022,09-11-2022,C/09/609567 / HA ZA 21-300,Civiel recht,Bodemzaak\nEerste aanleg - meervoudig,Beroep op volledige nadeelscompensatie (voor o...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
2,Rechtbank Den Haag,09-11-2022,09-11-2022,C/09/601949 / HA ZA 20-1067,Civiel recht,Bodemzaak\nEerste aanleg - meervoudig,Geschil over het uit de VVD-fractie zetten van...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
3,Rechtbank Den Haag,07-11-2022,08-11-2022,C/09/623033 / FA RK 21-8848,Personen- en familierecht,Beschikking,Vaststellen van een verlengde onderhoudsplicht...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
4,Rechtbank Rotterdam,04-11-2022,04-11-2022,C/10/645656 / KG ZA 22-837,Civiel recht,Kort geding,Kort geding; vordering tegen bestuurdersaanspr...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN


In [3]:
pd.read_table("../data/trial_small/metadata_out.csv", index_col = 0).head(5)

,index,Datum uitspraak:,Datum publicatie:,Rechtsgebieden:,Bijzondere kenmerken:,Vindplaatsen:,Inhoudsindicatie:,Formele relaties:
0,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Kort geding. Vordering om Staat te verbieden e...,NaN
1,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,"Bodemzaak, Eerste aanleg - meervoudig",Rechtspraak.nl,Beroep op volledige nadeelscompensatie (voor o...,NaN
2,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,"Bodemzaak, Eerste aanleg - meervoudig",Rechtspraak.nl,Geschil over het uit de VVD-fractie zetten van...,NaN
3,https://uitspraken.rechtspraak.nl/inziendocume...,07-11-2022,08-11-2022,Personen- en familierecht,Beschikking,Rechtspraak.nl,Vaststellen van een verlengde onderhoudsplicht...,NaN
4,https://uitspraken.rechtspraak.nl/inziendocume...,04-11-2022,04-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Kort geding; vordering tegen bestuurdersaanspr...,NaN


In [4]:
# Coalesce the columns for Beslissing and De Beslissing
text = pd.read_table("../data/trial_small/data.csv", index_col = 0)
text['De beslissing'] = text['De beslissing'].combine_first(text['Beslissing'])
text = text.drop(['Beslissing'], axis = 1)

# Coalesce the columns for De Beoordeling and De beoordeling van het geschil
text['De beoordeling'] = text['De beoordeling'].combine_first(text['De beoordeling van het geschil'])
text = text.drop(['De beoordeling van het geschil'], axis = 1)

In [5]:
# now write a function to extract the plaintiffs and defendants:
# This is what should be in the for loop over all urls
url = "https://uitspraken.rechtspraak.nl/#!/details?id=ECLI:NL:RBDHA:2022:11389&showbutton=true"

driver = webdriver.Chrome()
driver.get(url)

time.sleep(3)
page = driver.execute_script('return document.body.innerHTML')
soup = BeautifulSoup(page, 'html.parser')
driver.close()


In [6]:
# Extract plaintiffs and defendants
# Find the div with the text "The desired text"
desired_text = soup.find("p", text="in de zaak van").parent.parent
# Find the next div after the desired text
desired_div = desired_text.find_next_sibling("div")
plaintiffs = desired_div

desired_text = soup.find("p", text="tegen").parent.parent
desired_div = desired_text.find_next_sibling("div")
defendants = desired_div

In [7]:
# Extract the plaintiffs and defendants exactly (noise can be removed later)
aux.extract_text_until(defendants, "parablock").split("\n")
aux.extract_text_until(plaintiffs, "parablock").split('\n')

['',
 '1  [eiseres 1] , te [plaats 1] ,',
 '1',
 '2.\t[eiseres 2] B.V., te [plaats 1] ,',
 '[eiseres 2] B.V.',
 '3.\tSBR POWERHOUSE B.V., te Den Haag,',
 'SBR POWERHOUSE B.V.',
 '4.\tMYOBI B.V., te Den Haag,',
 'MYOBI B.V.',
 '']

In [8]:
# Extract their lawyers if present
defendants.find_all(text = lambda text: re.search(r'mr|advocaat|gemachtig', text))
plaintiffs.find_all(text = lambda text: re.search(r'mr|advocaat|gemachtig', text))

['advocaat mr. Ch.E. Koster te Den Haag, ']

In [9]:
# Bedrag?
## check conventie/reconventie
## loop through variables text instead of text['Het geschil'].iloc[i]
## set operator because otherwise numbers are being mentioned more than once, filter that out
geschil = text['Het geschil'].iloc[7]

## if reconventie
if re.search(r'reconventie', geschil):
    conventie = re.search(r".*reconventie", geschil).group(0)
    reconventie = re.search(r"reconventie.*", geschil).group(0)
    
    raw_conventie = re.findall(r"€ .*?\s", conventie)
    raw_reconventie = re.findall(r"€ .*?\s", reconventie)
    
    unique_numbers_conventie = set([aux.convert_money(x) for x in raw_conventie])
    unique_numbers_reconventie = set([aux.convert_money(x) for x in raw_reconventie])
    
## if no reconventie    
else: 
    raw_numbers = re.findall(r"€ .*?\s", geschil)
    numbers = [convert_money(x) for x in raw_numbers]
    unique_numbers = set(numbers)
    


In [10]:
# Volledig toegewezen/afgewezen?
conventie = re.search(r".*reconventie", geschil).group(0)
reconventie = re.search(r"reconventie.*", geschil).group(0)
    
raw_conventie = re.findall(r"€ .*?\s", conventie)
raw_reconventie = re.findall(r"€ .*?\s", reconventie)

unique_numbers_conventie = set([aux.convert_money(x) for x in raw_conventie])
unique_numbers_reconventie = set([aux.convert_money(x) for x in raw_reconventie])

In [11]:
unique_numbers_conventie

{555.71, 4307.07, 4862.78}

In [12]:
unique_numbers_reconventie

{498.69, 2871.38}

In [86]:
## two different things i can do
## 1. capture reconventie and conventie before splitting
## 2. looking at each point separately and judging then
## I implement 1
## pipeline as follows. Loop through all beslissingen
beslissing = text['De beslissing'].iloc[325]

lose = dict()
lose_conv = dict()
lose_reconv = dict()
money = dict()
money_conv = dict()
money_reconv = dict()

if re.search(r'reconventie', beslissing):
    # Split up conventie and reconventie part
    conventie, _, reconventie = clause.partition('reconventie')
    ### 2. loop through all clauses to find the numbers involved (conventie)
    clauses = re.split('\d\.\d\.', conventie)
    for i, clause in enumerate(clauses):
        lose_conv[i] = re.search(r"wijst.*vorder.*af|.*niet-ontvankelijk.*", clause)
        bedragen = re.findall(r"€ .*?\s", clause)
        numbers = [aux.convert_money(x) for x in bedragen]
        unique_numbers = set(numbers)
        money_conv[i] = unique_numbers
    ### 3. loop through all clauses to find the numbers involved (reconventie)
    clauses = re.split('\d\.\d\.', reconventie)
    for i, clause in enumerate(clauses):
        lose_reconv[i] = re.search(r"wijst.*vorder.*af|.*niet-ontvankelijk.*", clause)
        bedragen = re.findall(r"€ .*?\s", clause)
        numbers = [aux.convert_money(x) for x in bedragen]
        unique_numbers = set(numbers)
        money_reconv[i] = unique_numbers
               
else:
    clauses = re.split('\d\.\d\.', beslissing)
    for i, clause in enumerate(clauses):
        ### 3. Within loop, extract the first number after each veroordeling
        lose[i] = re.search(r"wijst.*vorder.*af|.*niet-ontvankelijk.*", clause)
        bedragen = re.findall(r"€ .*?\s", clause)
        numbers = [aux.convert_money(x) for x in bedragen]
        unique_numbers = set(numbers)
        money[i] = unique_numbers

# return lose, lose_conv, lose_reconv, money, money_conv, money_reconv
lose
money


{0: set(), 1: set(), 2: {1692.0}, 3: {85.0, 163.0}, 4: set()}

In [84]:
# Find stuff with reconventie in it to test
#text['De beslissing'].str.contains('reconventie')[50:100]


In [ ]:
# To do tomorrow: 
# Implement all of this stuff in a function using aux, and pay attention to the input:
# (one uses the URL, other uses the text)
# But can be implemented on merged DataFrame of the two